In [ ]:
import warnings

import numpy as np
import pandas as pd
import tqdm
#import umap  # depend de si python=3.9 ou pas
import umap.umap_ as umap
from lifelines import CoxPHFitter
from lifelines.utils import ConvergenceError
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.preprocessing import QuantileTransformer

In [ ]:
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
np.random.seed(42)

In [ ]:
path_ = "../features_with_clinical_data_KNN.csv"
num_bootstraps = 1000
NUMBER_OF_FEATURES = range(1, 11)
penalizer = [1e-5, 1e-4, 1e-3, 1e-2]

In [ ]:
df = pd.read_csv(path_)
df.drop(["weight"], inplace=True, axis=1)
df = df.astype(np.float32)
df.set_index("patient_id", inplace=True)
df.drop(["PFS"], inplace=True, axis=1)
df.rename(columns={"Follow-up Status": "FUS"}, inplace=True)

In [ ]:
last_cols = ["C" + str(i) for i in range(df.shape[1]-19)]
first_cols = ['MYC IHC', 'BCL2 IHC', 'BCL6 IHC', 'CD10 IHC', 'MUM1 IHC',
        'HANS', 'BCL6 FISH', 'MYC FISH', 'BCL2 FISH', 'Age', 'ECOG PS', 'LDH',
        'EN', 'Stage', 'IPI Score', 'IPI Risk Group (4 Class)', 'RIPI Risk Group', 'OS', 'FUS']

df.columns = first_cols + last_cols

In [ ]:
def VarianceMethod(df, n_feats):
    new_df = df[last_cols]
    std = []
    for col in new_df.columns:
        std.append(df[col].std())
    
    std = np.array(std)
    std_top_n = np.argsort(std)[-n_feats:]
    std_top_n_df = new_df.iloc[:, std_top_n]
    final_df = pd.concat([df[first_cols], std_top_n_df], axis=1)
    return final_df

In [ ]:
def PCAMethod(df, n_feats):
    new_df = df[last_cols]
    pca = PCA(n_components=n_feats)
    pca.fit(new_df)
    pca_transformed = pca.transform(new_df)
    pca_df = pd.DataFrame(data=pca_transformed, columns=['PC{}'.format(i+1) for i in range(n_feats)])
    final_df = np.hstack((df[first_cols], pca_df))
    final_df= pd.DataFrame(final_df, columns=list(df[first_cols].columns) + list(pca_df.columns))
    return final_df

In [ ]:
def ICAMethod(df, n_feats):
    new_df = df[last_cols]
    ica = FastICA(n_components=n_feats)
    ica.fit(new_df)
    ica_transformed = ica.transform(new_df)
    ica_df = pd.DataFrame(data=ica_transformed, columns=['IC{}'.format(i+1) for i in range(n_feats)])
    final_df = np.hstack((df[first_cols], ica_df))
    final_df= pd.DataFrame(final_df, columns=list(df[first_cols].columns) + list(ica_df.columns))
    return final_df

In [ ]:
def UMAPMethod(df, n_feats):
    new_df = df[last_cols]
    reducer = umap.UMAP(n_components=n_feats)
    reducer.fit(new_df)
    umap_transformed = reducer.transform(new_df)
    umap_df = pd.DataFrame(data=umap_transformed, columns=['UMAP{}'.format(i+1) for i in range(n_feats)])
    final_df = np.hstack((df[first_cols], umap_df))
    final_df = pd.DataFrame(final_df, columns=list(df[first_cols].columns) + list(umap_df.columns))
    return final_df

In [ ]:
def IsoMapMethod(df, n_feats):
    new_df = df[last_cols]
    iso = Isomap(n_components=n_feats)
    iso.fit(new_df)
    iso_transformed = iso.transform(new_df)
    iso_df = pd.DataFrame(data=iso_transformed, columns=['ISO{}'.format(i+1) for i in range(n_feats)])
    final_df = np.hstack((df[first_cols], iso_df))
    final_df= pd.DataFrame(final_df, columns=list(df[first_cols].columns) + list(iso_df.columns))
    return final_df

In [ ]:
def corrected_c_index(final_df, p):
    c_b_boot, c_b_orig = [], []
    bootstrap_size = len(final_df)

    for i in range(num_bootstraps):
        choices = np.random.choice(np.arange(0, len(final_df)), size=bootstrap_size, replace=True)
        new_df = final_df.iloc[choices]  #sample bootstrap replicate with replacement

        cph = CoxPHFitter(penalizer=p)
        cph.fit(new_df, duration_col='OS', event_col='FUS')  #fit on bootstrap

        c = cph.score(new_df, scoring_method="concordance_index")  #score on bootstrap
        c_b_boot.append(c)

        c = cph.score(final_df, scoring_method="concordance_index")  #score on original
        c_b_orig.append(c)

    c_b_orig = np.array(c_b_orig)
    c_b_boot = np.array(c_b_boot)
    error = np.mean(c_b_boot - c_b_orig)
    
    return error

In [ ]:
results = []
for n_feats in NUMBER_OF_FEATURES:
    print('n features:', 2**n_feats)
    for p in penalizer:
        for method, name in tqdm.tqdm((
            (VarianceMethod, 'variance'),
            (PCAMethod, 'pca'),
            (ICAMethod, 'ica'),
            (UMAPMethod, 'umap'),
            (IsoMapMethod, 'isomap'))):

            try:
                new_df = method(df, 2**n_feats)
            except (ValueError, TypeError) as e:
                results.append(dict(method=name, n_feats=n_feats, cindex=None, cindex_opt=None, error=None, hi=None, lo=None, ebm=None))
                continue
            
            bootstrap_size = len(new_df)

            qt = QuantileTransformer(n_quantiles=10, random_state=42)
            qt.fit(new_df)
            new_df = pd.DataFrame.from_records(qt.transform(new_df), columns=new_df.columns)

            cph = CoxPHFitter(penalizer=p)
            try:
                cph.fit(new_df, duration_col='OS', event_col='FUS')
            except ConvergenceError:
                results.append(dict(method=name, n_feats=n_feats, cindex=None, cindex_opt=None, error=None, hi=None, lo=None, ebm=None))
                continue
            
            c_main = cph.score(new_df, scoring_method="concordance_index")

            try:
                error = corrected_c_index(new_df, p)
            except ValueError:
                results.append(dict(method=name, n_feats=n_feats, cindex=c_main, cindex_opt=None, error=None, hi=None, lo=None, ebm=None))
                continue
            
            c_corrected = c_main - error

            c_indices = []
            for i in range(num_bootstraps):
                choices = np.random.choice(np.arange(0, len(new_df)), size=bootstrap_size, replace=True)
                boot_df = new_df.iloc[choices]

                c_index = cph.score(boot_df, scoring_method="concordance_index")
                c_indices.append(c_index)

            c_indices.sort()
            
            hi = c_indices[int(0.975*len(c_indices))]
            lo = c_indices[int(0.025*len(c_indices))]

            std_err = np.std(c_indices, ddof=1) / np.sqrt(num_bootstraps)
            t_value = 1.96
            ebm = t_value * std_err

            results.append(dict(method=name, n_feats=n_feats, cindex=c_main, cindex_opt=c_corrected, error=error, hi=hi, lo=lo, ebm=ebm))

In [ ]:
pd.DataFrame.from_dict(results).to_csv('../results.csv')

c:\Users\bilel.guetarni\anaconda3\envs\M1_project\lib\site-packages\sklearn\decomposition\_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.

ValueError: There are significant negative eigenvalues (0.0114487 of the maximum positive). Either the matrix is not PSD, or there was an issue while computing the eigendecomposition of the matrix.

c:\Users\bilel.guetarni\anaconda3\envs\M1_project\lib\site-packages\sklearn\decomposition\_fastica.py:589: UserWarning: n_components is too large: it will be set to 170
  warnings.warn(

TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

c:\Users\bilel.guetarni\anaconda3\envs\M1_project\lib\site-packages\sklearn\decomposition\_fastica.py:589: UserWarning: n_components is too large: it will be set to 170
  warnings.warn(